In [1]:
import notebook_styling as ns
ns.set_css()

# Multi-zone Single-Duct VAV System Sizing (Design or Peak Conditions) - Take One

*Example 19.8 from the book "Heating and Cooling of Buildings" by T. Agami Reddy, Jan F. Kreider, Peter S. Curtiss and Ari Rabl (3rd Edition, 2017)* 

A two-zone building is to be equipped with a VAV system having preheat- and reheat-coils for the load conditions in the table below. Size the supply fan, cooling coil, preheat coil, and reheat coil (or baseboard heating) for this building. Note that these will also apply to a CAV system under peak design conditions.

***Space loads***

<div class="my_table">
    <table>
        <tr>
            <th></th>
            <th>Zone A (Exterior)</th>
            <th>Zone B (Interior)</th>
        </tr>
        <tr>
            <td>Sensible peak summer design cooling load</td>
            <td>224,844 Btu/hr</td>
            <td>103,308 Btu/hr</td>
        </tr>
        <tr>
            <td>Latent peak summer design cooling load</td>
            <td>56,000 Btu/hr</td>
            <td>20,000 Btu/hr</td>
        </tr>
        <tr>
            <td>Sensible peak winter design load</td>
            <td>-143,000 Btu/hr</td>
            <td>49,092 Btu/hr</td>
        </tr>
        <tr>
            <td>Zone temperature</td>
            <td>75°F</td>
            <td>75°F</td>
        </tr>
    </table>
</div>

>Notice that interior zone B still requires cooling in winter.

***Design parameters***

1. summer design dry-and wet-bulb temperatures: 97°F and 76°F (resulting in 0.0144 lb<sub>w</sub>/lb<sub>a</sub> humidity ratio at peak conditions).
2. winter design temperature: 7°F and air assumed to be totally dry.
3. pressure drop in the supply system at full air-flow: 3.0 inWG.
4. fan efficiency: 60%.
5. return air fan air temperature rise: none.
6. ventilation airflow rate: 2400 ft<sup>3</sup>/min.
7. relative air humidity at cooling coil outlet: 85%.
8. temperature difference between supply air and zone air: 20°F (to ensure proper mixing).
9. maximum supply air temperature: 105°F (to avoid stratification).

***Sketch***

<center><img src="./images/airco_vav.png" alt="simplified sketch of plenum return single-duct VAV system" style="width:75%; height:75%;"></center>

***Assumptions***

- Ignore factors not included in the list given earlier, such as duct heat losses and gains. 
- The location is assumed to be at sea level.
- Rather than use enthalpy balances on the zones, the flows will be designed based on sensible heat loads (this is meant to simplify the problem). 
- Peak loads are coincident; no diversity adjustment is used.
- During peak heating, latent loads on the space are negligible. 
- During winter, room humidifiers are used to maintain the necessary humidity levels. 
- The supply airflow to the zones cannot be reduced to less than 60 % of the full-load design value by mass to ensure proper mixing of the cold supply air once it enters the space.

In [2]:
from hvac import Quantity
from hvac.fluids import HumidAir
from hvac.air_conditioning import (
    AirConditioningProcess,
    AirStream,
    AdiabaticMixing,
    Fan
)

Qty = Quantity

In [3]:
class Summer:
    pass


class Winter:
    pass

### Solution

### 1. Mass Flow Rate of Ventilation Air

The required outdoor air volume flow rate for ventilation is already determined at the start of the design process.

In [4]:
V_vent = Qty(2400, 'ft ** 3 / min')

Summer peak outdoor air condition:

In [5]:
Summer.outdoor_air = HumidAir(
    Tdb=Qty(97, 'degF'),
    Twb=Qty(76, 'degF')
)

Mass flow rate of ventilation air:

In [6]:
m_vent = Summer.outdoor_air.rho * V_vent

In [7]:
print(
    "--> mass flow rate of ventilation air = "
    f"{m_vent.to('lb / min'):~P.1f}"
)

--> mass flow rate of ventilation air = 167.2 lb/min


### **COOLING DESIGN DAY**

### 2. Zone Supply Air Flow Rates

Zone air temperature:

In [8]:
T_zone = Qty(75, 'degF')

Supply air temperature = zone air temperature - 20 °F:

In [9]:
Summer.T_supply = T_zone - Qty(20, 'delta_degF')

Relative air humidity at cooling coil exit = 85%. The humidity of the supply air remains unchanged:

In [10]:
Summer.supply_air = HumidAir(
    Tdb=Summer.T_supply,
    RH=Qty(85, 'pct')
)

Apply sensible heat balance to zone A to determine supply air mass flow rate to zone A:

In [11]:
zone_A = AirConditioningProcess(
    T_ai=Summer.T_supply,
    T_ao=T_zone,
    Q_sen=Qty(224_844, 'Btu / hr')
)
Summer.m_supply_zA = zone_A.m_da

Apply sensible heat balance to zone B to determine supply air mass flow rate to zone B:

In [12]:
zone_B = AirConditioningProcess(
    T_ai=Summer.T_supply,
    T_ao=T_zone,
    Q_sen=Qty(103_308, 'Btu / hr')
)
Summer.m_supply_zB = zone_B.m_da

Determine total mass flow rate and total volume flow rate of supply air to zones:

In [13]:
Summer.m_supply = Summer.m_supply_zA + Summer.m_supply_zB
Summer.V_supply = Summer.supply_air.v * Summer.m_supply

In [14]:
ns.display_list([
    "mass flow rate of supply air to zone A = "
    f"<mark><b>{Summer.m_supply_zA.to('lb / min'):~P.3f}</b></mark>",
    "mass flow rate of supply air to zone B = "
    f"<mark><b>{Summer.m_supply_zB.to('lb / min'):~P.3f}</b></mark>",
    "total mass flow rate of supply air to zones = "
    f"<mark><b>{Summer.m_supply.to('lb / min'):~P.3f}</b></mark>",
    "total volume flow rate of supply air to zones = "
    f"<mark><b>{Summer.V_supply.to('ft ** 3 / min'):~P.3f}</b></mark>"
])

### 3. Fan Mechanical Power

After determining the volume flow rates in the (see previous step 2), the total pressure drop in the supply air system can be calculated. The result is here 3 inWG:

In [15]:
dP_fan = Qty(3.0, 'inch_H2O_60F')

Once volume flow rate and system pressure drop are known, a fan can be selected. The fan efficiency is here 60 %:

In [16]:
eta_fan = Qty(60.0, 'pct')

The mechanical power taken up by the fan can now be calculated as:

In [17]:
W_fan = Summer.V_supply * dP_fan / eta_fan

In [18]:
print(f"--> mechanical power taken up by fan = {W_fan.to('hp'):~P.0f}")

--> mechanical power taken up by fan = 12 hp


### 4. Supply Air Temperature
This was already determined in step 2: `Summer.T_supply`.

### 5. Air Temperature at Cooling Coil Exit
To determine the required air temperature at the cooling coil exit, any heating of the supply air due to inefficiency of the supply fan, and due to any heat absorption between the fan and the zone outlets also need to be taken into account. The cooling coil exit coincides with the supply fan inlet. The condition of the supply air at the fan exit has already been determined in step 2. The air condition at the inlet of the fan can be computed with:

In [19]:
fan = Fan(
    air_out=Summer.supply_air,
    eta_fan=eta_fan,
    dP_fan=Qty(3.0, 'inch_H2O_60F')
)

The temperature rise across the fan follows from:

In [20]:
dT_fan = fan.air_out.Tdb - fan.air_in.Tdb

In [21]:
print(f"--> fan temperature rise = {dT_fan.to('delta_degF'):~P.0f}")

--> fan temperature rise = 2 Δ°F


Duct heat gain is neglected here:

In [22]:
dT_duct_gain = Qty(0.0, 'delta_degF')

The required air temperature at the cooling coil exit follows from: 

In [23]:
Summer.T_cold = Summer.T_supply - dT_fan - dT_duct_gain
Summer.cooled_air = HumidAir(
    Tdb=Summer.T_cold,
    RH=Qty(85.0, 'pct')
)

In [24]:
ns.display_list([
    "required cooling coil leaving temperature = "
    f"<mark><b>{Summer.T_cold.to('degF'):~P.0f}</b></mark>"
])

### 6. Return Air Condition
Return duct heat gain is neglected here. The return air dry-bulb temperature for each zone is `T_zone`. The return air humidity of each zone is determined by the space condition line and the given zone temperature:

**Zone A**

In [25]:
Q_sen_zA = Qty(224_844, 'Btu / hr')
Q_lat_zA = Qty(56_000, 'Btu / hr')
SHR_zA = Q_sen_zA / (Q_sen_zA + Q_lat_zA)

zone_A = AirConditioningProcess(
    air_in=Summer.supply_air,
    T_ao=T_zone,
    SHR=SHR_zA
)
Summer.return_air_zA = zone_A.air_out

In [26]:
print(
    "- dry-bulb temperature of return air from zone A = "
    f"{Summer.return_air_zA.Tdb.to('degF'):~P.0f}",
    "- relative humidity of return air from zone A = "
    f"{Summer.return_air_zA.RH.to('pct'):~P.0f}",
    sep='\n'
)

- dry-bulb temperature of return air from zone A = 75 °F
- relative humidity of return air from zone A = 48 %


**Zone B**

In [27]:
Q_sen_zB = Qty(103_308, 'Btu / hr')
Q_lat_zB = Qty(20_000, 'Btu / hr')
Q_tot_zB = Q_sen_zB + Q_lat_zB
SHR_zB = Q_sen_zB / Q_tot_zB

zone_B = AirConditioningProcess(
    air_in=Summer.supply_air,
    T_ao=T_zone,
    SHR=SHR_zB
)
Summer.return_air_zB = zone_B.air_out

In [28]:
print(
    "- dry-bulb temperature of return air from zone B = "
    f"{Summer.return_air_zB.Tdb.to('degF'):~P.0f}",
    "- relative humidity of return air from zone B = "
    f"{Summer.return_air_zB.RH.to('pct'):~P.0f}",
    sep='\n'
)

- dry-bulb temperature of return air from zone B = 75 °F
- relative humidity of return air from zone B = 47 %


**Return Air to Mixing Chamber**

In [29]:
mix_return_air = AdiabaticMixing(
    in1=AirStream(
        state=Summer.return_air_zA,
        m_da=Summer.m_supply_zA
    ),
    in2=AirStream(
        state=Summer.return_air_zB,
        m_da=Summer.m_supply_zB
    ),
    out=AirStream(m_da=Summer.m_supply)
)
Summer.return_air = mix_return_air.stream_out.state

In [30]:
print(
    "- dry-bulb temperature of return air to mixing chamber = "
    f"{Summer.return_air.Tdb.to('degF'):~P.0f}",
    "- relative humidity of return air to mixing chamber = "
    f"{Summer.return_air.RH.to('pct'):~P.0f}",
    sep='\n'
)

- dry-bulb temperature of return air to mixing chamber = 75 °F
- relative humidity of return air to mixing chamber = 48 %


### 7. Adiabatic Mixing of Ventilation and Return Air

In [31]:
mixing_chamber = AdiabaticMixing(
    in1=AirStream(
        state=Summer.outdoor_air,
        m_da=m_vent
    ),
    in2=AirStream(
        state=Summer.return_air,
        m_da=Summer.m_supply - m_vent
    ),
    out=AirStream(m_da=Summer.m_supply)
)
Summer.mixed_air = mixing_chamber.stream_out.state

In [32]:
ns.display_list([
    "dry-bulb temperature of mixed air = "
    f"<mark><b>{Summer.mixed_air.Tdb.to('degF'):~P.0f}</b></mark>",
    "relative humidity of mixed_air = "
    f"<mark><b>{Summer.mixed_air.RH.to('pct'):~P.0f}</b></mark>"
])

### 9. Cooling Coil Load

In [33]:
cooling_coil = AirConditioningProcess(
    air_in=Summer.mixed_air,
    air_out=Summer.cooled_air,
    m_da=Summer.m_supply,
    h_w=Qty(0.0, 'Btu / lb')
)
Summer.Q_cc = cooling_coil.Q
Summer.Q_cc_sen = cooling_coil.Q_sen
Summer.Q_cc_lat = cooling_coil.Q_lat

In [34]:
ns.display_list([
    "total cooling coil load = "
    f"<mark><b>{Summer.Q_cc.to('Btu / hr'):~P.0f}</b></mark> "
    f"({Summer.Q_cc.to('refrigeration_ton'):~P.2f})",
    "sensible cooling coil load = "
    f"<mark><b>{Summer.Q_cc_sen.to('Btu / hr'):~P.0f}</b></mark> "
    f"({Summer.Q_cc_sen.to('refrigeration_ton'):~P.2f})",
    "latent cooling coil load = "
    f"<mark><b>{Summer.Q_cc_lat.to('Btu / hr'):~P.0f}</b></mark> "
    f"({Summer.Q_cc_lat.to('refrigeration_ton'):~P.2f})"
])

### **HEATING DESIGN DAY**

### 10. Preheat Coil Peak Load
The peak load on the preheating coil is determined based on the load needed to heat the cold outdoor ventilation air on the winter design day to the nominal coil outlet temperature (`Summer.T_cold`).

In [35]:
Winter.T_outdoor = Qty(7, 'degF')
Winter.outdoor_air = HumidAir(
    Tdb=Winter.T_outdoor,
    RH=Qty(0, 'pct')
)  # fully dry air

m_vent = Winter.outdoor_air.rho * V_vent

preheat_coil = AirConditioningProcess(
    T_ai=Winter.T_outdoor,
    T_ao=Summer.T_cold,
    m_da=m_vent
)
Winter.Q_ph_peak = preheat_coil.Q_sen

In [36]:
ns.display_list([
    "preheat coil peak load = "
    f"<mark><b>{Winter.Q_ph_peak.to('Btu / hr'):~P.0f}</b></mark>"
])

### 11. Zone Supply Air Flow Rates

**Zone A (requires heating)**<br>
At first, take the maximum allowable supply air temperature of 105°F to determine mass flow rate:

In [37]:
Winter.T_supply = Qty(105, 'degF')

zone_A = AirConditioningProcess(
    T_ai=Winter.T_supply,
    T_ao=T_zone,
    Q_sen=Qty(-143_000, 'Btu / hr')
)
Winter.m_supply_zA = zone_A.m_da

In [38]:
print(
    "--> supply air mass flow rate to zone A = "
    f"{Winter.m_supply_zA.to('lb / min'):~P.2f}"
)

--> supply air mass flow rate to zone A = 326.10 lb/min


Recall the stipulation that mass flow rate cannot be less than 60% of the peak air flow.

In [39]:
Winter.m_supply_zA = max(
    zone_A.m_da,
    0.6 * Summer.m_supply_zA
)

In [40]:
ns.display_list([
    "supply air mass flow rate to zone A = "
    f"<mark><b>{Winter.m_supply_zA.to('lb / min'):~P.2f}</b></mark>"
])

**Zone B (requires cooling)**<br>
At first, take the minimum supply air temperature of 55°F to determine mass flow rate. Next, check again for the condition that mass flow rate cannot be less than 60% of the peak air flow:

In [41]:
zone_B = AirConditioningProcess(
    T_ai=Qty(55.0, 'degF'),
    T_ao=T_zone,
    Q_sen=Qty(49_092, 'Btu / hr')
)

Winter.m_supply_zB = max(
    zone_B.m_da,
    0.6 * Summer.m_supply_zB
)

In [42]:
ns.display_list([
    "supply air mass flow rate to zone B = "
    f"<mark><b>{Winter.m_supply_zB.to('lb / min'):~P.2f}</b></mark>"
])

**Total supply air mass flow rate**

In [43]:
Winter.m_supply = Winter.m_supply_zA + Winter.m_supply_zB

In [44]:
ns.display_list([
    "total supply air mass flow rate = "
    f"<mark><b>{Winter.m_supply.to('lb / min'):~P.2f}</b></mark>"
])

### 12. Supply Air Temperature

**Zone A**

In [45]:
zone_A = AirConditioningProcess(
    T_ao=T_zone,
    m_da=Winter.m_supply_zA,
    Q_sen=Qty(-143_000, 'Btu / hr')
)
Winter.T_supply_zA = zone_A.T_ai

In [46]:
print(
    "--> required supply air temperature for zone A: "
    f"{Winter.T_supply_zA.to('degF'):~P.1f}"
)

--> required supply air temperature for zone A: 96.2 °F


**Zone B**

In [47]:
zone_B = AirConditioningProcess(
    T_ao=T_zone,
    m_da=Winter.m_supply_zB,
    Q_sen=Qty(49_092, 'Btu / hr')
)
Winter.T_supply_zB = zone_B.T_ai

In [48]:
print(
    "--> required supply air temperature for zone B: "
    f"{Winter.T_supply_zB.to('degF'):~P.1f}"
)

--> required supply air temperature for zone B: 59.2 °F


**Final supply air temperature**<br>
The final supply air temperature must be the smallest required supply air temperature of all zones.

In [49]:
Winter.T_supply = min(Winter.T_supply_zA, Winter.T_supply_zB)
Winter.supply_air = HumidAir(
    Tdb=Winter.T_supply,
    RH=Qty(0, 'pct')
)

In [50]:
ns.display_list([
    "required supply air temperature = "
    f"<mark><b>{Winter.T_supply.to('degF'):~P.1f}</b></mark>"
])

### 13. Return Air Condition

In [51]:
mix_return_air = AdiabaticMixing(
    in1=AirStream(
        state=HumidAir(
            Tdb=T_zone,
            RH=Qty(0.0, 'pct')
        ),
        m_da=Winter.m_supply_zA
    ),
    in2=AirStream(
        state=HumidAir(
            Tdb=T_zone,
            RH=Qty(0.0, 'pct')
        ),
        m_da=Winter.m_supply_zB
    ),
    out=AirStream(m_da=Winter.m_supply)
)
Winter.return_air = mix_return_air.stream_out.state

### 14. Adiabatic Mixing of Ventilation and Return Air

In [52]:
mixing_chamber = AdiabaticMixing(
    in1=AirStream(
        state=Winter.outdoor_air,
        m_da=m_vent
    ),
    in2=AirStream(
        state=Winter.return_air,
        m_da=Winter.m_supply - m_vent
    ),
    out=AirStream(m_da=Winter.m_supply)
)
Winter.mixed_air = mixing_chamber.stream_out.state

In [53]:
print(
    "- dry-bulb temperature of mixed air = "
    f"{Winter.mixed_air.Tdb.to('degF'):~P.1f}",
    "- relative humidity of mixed air = "
    f"{Winter.mixed_air.RH.to('pct'):~P.0f}",
    sep='\n'
)

- dry-bulb temperature of mixed air = 54.4 °F
- relative humidity of mixed air = 0 %


### 15. Cooling coil load/Preheat coil load

**Cooling coil leaving temperature**

Temperature rise of air due to fan:

In [54]:
fan = Fan(
    air_out=Winter.supply_air,
    eta_fan=eta_fan,
    dP_fan=Qty(3.0, 'inch_H2O_60F')
)
dT_fan = fan.air_out.Tdb - fan.air_in.Tdb

Duct heat gain:

In [55]:
dT_duct_gain = Qty(0.0, 'delta_degF')

Required air temperature at cooling coil exit:

In [56]:
Winter.T_cold = Winter.T_supply - dT_fan - dT_duct_gain

In [57]:
ns.display_list([
    "required cooling coil leaving temperature = "
    f"<mark><b>{Winter.T_cold.to('degF'):~P.1f}</b></mark>"
])

Note that the required air temperature at the cooling coil exit is higher than the temperature of the mixed air. So, instead of cooling, the air must be heated in the preheating coil.

**Preheat coil load**

In [58]:
preheat_coil = AirConditioningProcess(
    T_ai=Winter.mixed_air.Tdb,
    T_ao=Winter.T_cold,
    m_da=Winter.m_supply
)
Winter.Q_ph = preheat_coil.Q_sen

In [59]:
ns.display_list([
    "preheat coil load = "
    f"<mark><b>{Winter.Q_ph.to('Btu / hr'):~P.0f}</b></mark>"
])

### 16. Reheat Coil Loads

**Zone A**

In [60]:
reheater_zA = AirConditioningProcess(
    T_ai=Winter.T_supply,
    T_ao=Winter.T_supply_zA,
    m_da=Winter.m_supply_zA
)
Winter.Q_rh_zA = reheater_zA.Q_sen

In [61]:
ns.display_list([
    "reheat coil load of zone A = "
    f"<mark><b>{Winter.Q_rh_zA.to('Btu / hr'):~P.0f}</b></mark>"
])

**Zone B**

In [62]:
reheater_zB = AirConditioningProcess(
    T_ai=Winter.T_supply,
    T_ao=Winter.T_supply_zB,
    m_da=Winter.m_supply_zB
)
Winter.Q_rh_zB = reheater_zB.Q_sen

In [63]:
ns.display_list([
    f"reheat coil load of zone B = "
    f"<mark><b>{Winter.Q_rh_zB.to('Btu / hr'):~P.0f}</b></mark>"
])

**Total peak heating requirement**

In [64]:
Winter.Q_heat_tot = Winter.Q_ph_peak + Winter.Q_rh_zA + Winter.Q_rh_zB

In [65]:
ns.display_list([
    "total peak heating load = "
    f"<mark><b>{Winter.Q_heat_tot.to('Btu / hr'):~P.3f}</b></mark>"
])

This heat rate must be increased by pickup loads from night setback, safety factors, and piping losses in order to size the coils and the boiler.